################################################################################
> # **Install Dependencies**
1. pyrosetta
2. colabdesign
################################################################################

In [1]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta as pr
from pyrosetta import *
from pyrosetta.rosetta.protocols.rosetta_scripts import *
from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.protocols.simple_moves import AlignChainMover
import os
import re
import numpy as np

Mounted at /content/google_drive

Note that USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE.
See https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.md or email license@uw.edu for details.

Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2023.40+release.96fa3c54b9f-cp310-cp310-linux_x86_64.whl




In [2]:
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/ohuelab/ColabDesign-cyclic-binder.git@cyc_binder")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.shared.utils import copy_dict

In [3]:
# initialize pyrosetta
init('-corrections::beta_nov16 -detect_disulf false', extra_options="-run:preserve_header true")

PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2023.40+release.96fa3c54b9f6e46b5bfdeb343261506bc4369812 2023-10-04T07:52:04] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r359 2023.40+release.96fa3c54b9f 96fa3c54b9f6e46b5bfdeb343261506bc4369812 http://www.pyrosetta.org 2023-10-04T07:52:04
core.init: command: PyRosetta -corrections::beta_nov16 -detect_disulf false -run:preserve_header true -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=848037982 seed_offset=0 real_seed=848037982
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=848037982 RG_type=mt19937


In [4]:
# download a pdb file to test
!wget https://raw.githubusercontent.com/Bop2000/DOTS/setup-install/SelfDriving_Virtual_Labs/Cyclic_Peptide_Design/4ib5.pdb

--2024-04-27 08:03:16--  https://raw.githubusercontent.com/Bop2000/DOTS/setup-install/SelfDriving_Virtual_Labs/Cyclic_Peptide_Design/4ib5.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 466487 (456K) [text/plain]
Saving to: ‘4ib5.pdb’

4ib5.pdb            100%[===================>] 455.55K  --.-KB/s    in 0.04s   

2024-04-27 08:03:16 (12.7 MB/s) - ‘4ib5.pdb’ saved [466487/466487]



################################################################################
> # **Introduction**
> The notebook is divided into 3 major parts :

*   **Part I** : Use pyrosetta to calculate metrics for Alphafold predicted complex
*   **Part II** : Define DOTS algorithm
*   **Part III** : Optimization using DOTS

################################################################################

################################################################################
> # **Part - I**

*   Define pyrosetta interface score function
*   Use Alphafold to predict the structure of complex

################################################################################

In [5]:
def score_interface(pdb_file):
    # load pose
    pose = pr.pose_from_pdb(pdb_file)

    # analyze interface statistics
    iam = InterfaceAnalyzerMover()
    iam.set_interface("A_B")
    scorefxn = pr.get_fa_scorefxn()
    iam.set_scorefunction(scorefxn)
    iam.set_compute_packstat(True)
    iam.set_compute_interface_energy(True)
    #iam.set_compute_interface_delta_hbond_unsat(True)
    iam.set_calc_dSASA(True)
    iam.set_calc_hbond_sasaE(True)
    iam.set_compute_interface_sc(True)
    iam.set_pack_separated(True)
    iam.apply(pose)

    # retrieve statistics
    interfacescore = iam.get_all_data()
    interface_sc = interfacescore.sc_value # shape complementarity
    interface_nres = iam.get_num_interface_residues() # number of interface residues
    interface_interface_hbonds = interfacescore.interface_hbonds # number of interface H-bonds
    interface_dG = iam.get_interface_dG() # interface dG
    interface_dSASA = iam.get_interface_delta_sasa() # interface dSASA (interface surface area)
    interface_packstat = iam.get_interface_packstat() # interface pack stat score
    interface_dG_SASA_ratio = interfacescore.dG_dSASA_ratio * 100
    interface_scores = {
    'interface_sc': interface_sc,
    'interface_packstat': interface_packstat,
    'interface_dG': interface_dG,
    'interface_dSASA': interface_dSASA,
    'interface_dG_SASA_ratio': interface_dG_SASA_ratio,
    'interface_nres': interface_nres,
    'interface_interface_hbonds': interface_interface_hbonds,
    }
    return interface_scores

In [6]:
def add_cyclic_offset(self, bug_fix=True):
    '''add cyclic offset to connect N and C term'''
    def cyclic_offset(L):
        i = np.arange(L)
        ij = np.stack([i,i+L],-1)
        offset = i[:,None] - i[None,:]
        c_offset = np.abs(ij[:,None,:,None] - ij[None,:,None,:]).min((2,3))
        if bug_fix:
            a = c_offset < np.abs(offset)
            c_offset[a] = -c_offset[a]
        return c_offset * np.sign(offset)
    idx = self._inputs["residue_index"]
    offset = np.array(idx[:,None] - idx[None,:])

    if self.protocol == "binder":
        c_offset = cyclic_offset(self._binder_len)
        offset[self._target_len:,self._target_len:] = c_offset
    self._inputs["offset"] = offset

In [7]:
def int2aa(seq):
    aacode = {
        "0": "A",
        "1": "R",
        "2": "N",
        "3": "D",
        "4": "C",
        "5": "Q",
        "6": "E",
        "7": "G",
        "8": "H",
        "9": "I",
        "10": "L",
        "11": "K",
        "12": "M",
        "13": "F",
        "14": "P",
        "15": "S",
        "16": "T",
        "17": "W",
        "18": "Y",
        "19": "V"
    }
    aa = [aacode[str(int(i))] for i in seq]
    return "".join(aa)

In [8]:
def set_model(binder, pdb='4ib5.pdb'):
    target_chain = "A" # Chain id of protein
    if pdb == '1sfi.pdb':
        target_hotspot = "40,41,57,97,99,175,192,195"  # define the interface hotspot
    elif pdb == '1sld.pdb':
        target_hotspot = "25,84,110"
    elif pdb == '1smf.pdb':
        target_hotspot = "39-42,57,60,94,96,99,151,189-193,195,213-216,219,210,226"
    elif pdb == '3p72.pdb':
        target_hotspot = "81,106,128,130,152,230,234,235,236"
    if pdb == '3zgc.pdb':
        target_hotspot = "334,363,364,380,382,415,483,508,525,530,555,556,572,574"
    elif pdb == '4ib5.pdb':
        target_hotspot = "36,37,39-42,52,54,57,67,69,71,101,103,106,108,110,112"
    elif pdb == '4kel.pdb':
        target_hotspot = "40,41,57,98,99,151,174,175,192,193,195,215,216,218"
    elif pdb == '5h5q.pdb':
        target_hotspot = "33,43,45,53,142"
    elif pdb == '5tu6.pdb':
        target_hotspot = "69,71,118,119,220,269,271"
    elif pdb == '6d40.pdb':
        target_hotspot = "603,719,738,761-763"
    elif pdb == '6u6k.pdb':
        target_hotspot = "81,82,91-94,143-146,149"
    elif pdb == '6vxy.pdb':
        target_hotspot = "175,192,219"
    elif pdb == '7ezw.pdb':
        target_hotspot = "46-48,52,56,60,88,90-92,94,100-103,112,153,155,157,166"
    elif pdb == '7k2j.pdb':
        target_hotspot = "334,363,364,380,382,414,415,483,508,509,525,530,555,556,572,577,602,603"
    else:
        return NotImplementedError
    if target_hotspot == "": target_hotspot = None
    target_flexible = False # allow backbone of target structure to be flexible


    binder_len = None # length of binder to hallucination
    binder_seq = binder
    binder_seq = re.sub("[^A-Z]", "", binder_seq.upper())
    if len(binder_seq) > 0:
        binder_len = len(binder_seq)
    else:
        binder_seq = None # if defined, will initialize design with this sequence
    # model config
    use_multimer = True # use alphafold-multimer for design
    num_recycles = 6 # ["0", "1", "3", "6"]
    num_models = "1" # ["1", "2", "3", "4", "5", "all"]
    num_models = 5 if num_models == "all" else int(num_models) # number of trained models to use during optimization

    x = {
        "pdb_filename":pdb,
        "chain":target_chain,
        "binder_len":binder_len,
        "hotspot":target_hotspot,
        "use_multimer":use_multimer,
        "rm_target_seq":target_flexible
        }

    if "x_prev" not in dir() or x != x_prev:
        clear_mem()
        model = mk_afdesign_model(
        protocol="binder",
        use_multimer=x["use_multimer"],
        num_recycles=num_recycles,
        recycle_mode="sample",) # Here is the dir of alphafold params

        model.prep_inputs(
        **x,
        ignore_missing=False
        )
        x_prev = copy_dict(x)
        binder_len = model._binder_len
        add_cyclic_offset(model, bug_fix=True)
        model.restart(seq=binder_seq)
    return model

In [9]:
def get_value(seq_folder, array, pdb='4ib5.pdb'):
    binder_seq = int2aa(array)
    model = set_model(binder_seq, pdb)
    model.predict()
    model.save_pdb(f"./{seq_folder}/{binder_seq}.pdb")
    values = score_interface(f"./{seq_folder}/{binder_seq}.pdb")
    return values



################################# End of Part I ################################

################################################################################
> # **Part - II**

*   Define the DOTS alghorithm

################################################################################

In [12]:
import random
from tqdm import tqdm
from abc import ABC, abstractmethod
from collections import defaultdict
import math
from collections import namedtuple
import time

In [13]:
pdb = '4ib5.pdb' # Define the pdb file. Here take the 4ib5 as an example
seq_len = 13 # Define the length of cyclic peptide here.
seq_folder = './DOTS_peptide_' + pdb[:4] + '_' + time.strftime("%Y-%m-%d_%H-%M", time.localtime())
if not os.path.exists(seq_folder):
# If it doesn't exist, create it
    os.makedirs(seq_folder)

In [15]:
native_value = score_interface(pdb)
print(native_value)
print('nature target: ', float(native_value['interface_sc']) * float(native_value['interface_dSASA']) / 100)

core.import_pose.import_pose: File '4ib5.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLN:CtermProteinFull 329
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLY:CtermProteinFull 342
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: beta_nov16.wts
protocols.analysis.InterfaceAnalyzerMover: Repacking separated Pose before running calculations
protocols.analysis.InterfaceAnalyzerMover: Using explicit constructor
protocols.analysis.InterfaceAnalyzerMover: Using interface constructor
protocols.analysis.InterfaceAnalyzerMover: [ WARNING ] In InterfaceAnalyzerMover, calculator Sasa_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: [ WARNING ] In InterfaceAnalyzerMover, calculator InterfaceNeighborDefinition_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: [ WARNING ] In 

In [21]:
class DOTS:
    "DOTS"
    def __init__(self, exploration_weight=1):
        self.Q = defaultdict(int)  # total reward of each node
        self.N = defaultdict(int)  # total visit count for each node
        self.Nstay = 0
        self.children = dict()  # children of each node
        self.exploration_weight = exploration_weight

    def choose(self, node):
        "Choose the best successor of node. (Choose a move in the game)"
        if node not in self.children:
            print('not seen before, randomly sampled!')
            return node.find_random_child()

        print(f'number of visit is {self.N[node]}')
        log_N_vertex = math.log(self.N[node])
        def uct(n):
            "modified Upper confidence bound for trees"
            uct_value = n.value + self.exploration_weight * math.sqrt(
                log_N_vertex / (self.N[n]+1))
            return uct_value
        if (self.Nstay + 1) % 4 == 0:
            # if one node stay too long, then get more children node.
            action = [p for p in range(0, len(node.tup))]
            self.children[node] = self.children[node] | node.find_children(action)
        media_node = max(self.children[node], key=uct)#self._uct_select(node)
        rand_index = random.randint(0, len(list(self.children[node]))-1)
        node_rand = list(self.children[node])[rand_index]
        print(f'uct of the node is{uct(node)} ')
        if uct(media_node) > uct(node):
            print(f'better uct media node : {uct(media_node)}')
            print(f'better value media node : {media_node.value}')
            print('media_node: ', media_node)
            print('node_rand: ', node_rand)
            return media_node, media_node.value, node_rand, node_rand.value
        self.Nstay += 1
        print('node stays!', self.Nstay)
        print('node: ', node)
        print('node_rand: ', node_rand)
        return node, node.value, node_rand, node_rand.value

    def do_rollout(self, node):
        "Make the tree one layer better. (Train for one iteration.)"
        path = self._select(node)
        leaf = path[-1]
        self._expand(leaf)
        reward = self._simulate(leaf)
        self._backpropagate(path, reward)

    def _select(self, node):
        "Find an unexplored descendent of `node`"
        path = []
        while True:
            path.append(node)
            if node not in self.children or not self.children[node]:
                # node is either unexplored or terminal
                return path
            unexplored = self.children[node] - self.children.keys()
            def evaluate(n):
                return n.value
            if unexplored:
                path.append(max(unexplored, key=evaluate))#
                return path
            node = self._uct_select(node)  # descend a layer deeper

    def _expand(self, node):
        "Update the `children` dict with the children of `node`"
        if node in self.children:
            return  # already expanded
        action = [p for p in range(0, len(node.tup))]
        self.children[node] = node.find_children(action)

    def _simulate(self, node):
        "Returns the reward for a random simulation (to completion) of `node`"
        reward = node.value
        return reward


    def _backpropagate(self, path, reward):
        "Send the reward back up to the ancestors of the leaf"
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward

    def _uct_select(self, node):
        "Select a child of node, balancing exploration & exploitation"

        # All children of node should already be expanded:
        assert all(n in self.children for n in self.children[node])

        log_N_vertex = math.log(self.N[node])
        def uct(n):
            "Upper confidence bound for trees"
            uct_value = n.value + self.exploration_weight * math.sqrt(
                log_N_vertex / (self.N[n]+1))
            return uct_value
        uct_node = max(self.children[node], key=uct)
        print(f'node with max uct is:{uct_node}')
        return uct_node


In [22]:
class Node(ABC):
    """
    A representation of a single board state.
    DOTS works by constructing a tree of these Nodes.
    Could be e.g. a chess or checkers board state.
    """

    @abstractmethod
    def find_children(self):
        "All possible successors of this board state"
        return set()

    @abstractmethod
    def find_random_child(self):
        "Random successor of this board state (for more efficient simulation)"
        return None

    @abstractmethod
    def is_terminal(self):
        "Returns True if the node has no children"
        return True

    @abstractmethod
    def reward(self):
        "Assumes `self` is terminal node. 1=win, 0=loss, .5=tie, etc"
        return 0

    @abstractmethod
    def __hash__(self):
        "Nodes must be hashable"
        return 123456789

    @abstractmethod
    def __eq__(node1, node2):
        "Nodes must be comparable"
        return True

In [23]:
_OT = namedtuple("opt_task", "tup value terminal") # tup is the initial seq
class opt_task(_OT, Node):
    def find_children(board,action):
        if board.terminal:
            return set()
        all_tup=[]
        for index in action:
            tup = list(board.tup)
            flip = random.randint(0,5)
            if flip <= 3:
                while True:
                    mutation = random.randint(0,19)
                    if mutation != tup[index]:
                        break
                tup[index] = mutation
            elif flip:
                while True:
                    new_tup = [random.randint(0, 19) for _ in range(seq_len)]
                    if new_tup != tup:
                        break
                tup = new_tup
            print(tup, int2aa(tup))
            all_tup.append(tup)

        all_value = []
        for seq in all_tup:
            metrics = get_value(seq_folder, seq, pdb)
            all_value.append(float(metrics['interface_sc']) * float(metrics['interface_dSASA']) / 100)
        is_terminal=False
        return {opt_task(tuple(t), v, is_terminal) for t, v in zip(all_tup, all_value)}

    def find_random_child(board):
        pass

    def find_uct_child(board, action):
        pass

    def reward(board):
        pass

    def is_terminal(board):
        return board.terminal






################################ End of Part II ################################

################################################################################
> # **Part - III**

*   Optimization using DOTS

################################################################################


Input description:
*   initial_point: initial node for DOTS

Output description:

*   X_top: Best sequences of one round optimazation
*   y_top: The corresponding target values

In [24]:
def most_visit_node(tree_ubt, initial_X, top_n):
    N_visit = tree_ubt.N
    childrens = [i for i in tree_ubt.children]
    children_N = []
    X_top = []
    y_top = []
    for child in childrens:
        child_tup = np.array(child.tup)
        children_N.append(N_visit[child])
        X_top.append(child_tup)
        y_top.append(child.value)
    children_N = np.array(children_N)
    X_top = np.array(X_top)
    y_top = np.array(y_top)
    ind = np.argpartition(children_N, -top_n)[-top_n:]
    X_topN = X_top[ind]
    y_topN = y_top[ind]
    return X_topN, y_topN

In [25]:
def single_run(initial_X,top_n):
    metrics = get_value(seq_folder, initial_X, pdb)
    values = float(metrics['interface_sc']) * float(metrics['interface_dSASA']) / 100
    print(initial_X, values)
    exp_weight = 0.8 * values
    board_uct = opt_task(tup=tuple(initial_X), value=values, terminal=False)
    rollout_round = 15
    tree_ubt = DOTS(exploration_weight=exp_weight)
    boards = []
    board_values = []

    for i in tqdm(range(0, rollout_round)):
        print(i)
        tree_ubt.do_rollout(board_uct)
        board_uct, board_uct_value, board_rand, board_rand_value = tree_ubt.choose(board_uct)
        boards.append(list(board_uct.tup))
        board_values.append(board_uct_value)
        boards.append(list(board_rand.tup))
        board_values.append(board_rand_value)

    new_x = []
    new_pred = []
    boards = np.array(boards)

    for i,j in zip(boards,board_values):
        temp_x = np.array(i)
        new_pred.append(j)
        new_x.append(temp_x)
    new_x= np.array(new_x)
    new_pred = np.array(new_pred)
    temp = np.concatenate((new_x, new_pred.reshape(-1, 1)), axis=1)
    temp = np.unique(temp, axis=0)
    new_x = temp[:, :-1]
    new_pred = temp[:, -1]

    ind = np.argpartition(new_pred, -top_n)[-top_n:]
    top_x =  new_x[ind]
    top_y = new_pred[ind]
    print('top x: ',top_x)
    print('top y: ', top_y)

    X_most_visit, y_topN=  most_visit_node(tree_ubt, initial_X, 1)
    X_next = np.concatenate([top_x, X_most_visit])
    y_next = np.concatenate([top_y, y_topN])
    y_next = np.array(y_next)
    return X_next, y_next

In [ ]:
def run():
    x_current_top = [random.randint(0, 19) for _ in range(seq_len)] # random start sequence
    y_top=[]
    X_top=[]

    x, y_0 = single_run(x_current_top,2)
    y_top.append(y_0)
    X_top.append(x)

    for i in x:
        new_x, new_y = single_run(i, 2) # the new root node is the sequence with highest value in last round
        X_top.append(new_x)
        y_top.append(new_y)

    return X_top,y_top

X_top,y_top = run()
print(X_top)
print(y_top)

predict models [0] recycles 6 hard 1 soft 0 temp 1 loss 3.97 i_con 3.90 plddt 0.35 ptm 0.91 i_ptm 0.41
core.import_pose.import_pose: File '././DOTS_peptide_4ib5_2024-04-27_08-06/TFYFDRFGDINMY.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLN:CtermProteinFull 329
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue TYR:CtermProteinFull 342
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: beta_nov16.wts
protocols.analysis.InterfaceAnalyzerMover: Repacking separated Pose before running calculations
protocols.analysis.InterfaceAnalyzerMover: Using explicit constructor
protocols.analysis.InterfaceAnalyzerMover: Using interface constructor
protocols.analysis.InterfaceAnalyzerMover: [ WARNING ] In InterfaceAnalyzerMover, calculator Sasa_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: [ WARNING ] In InterfaceAnalyzerMover, calculat

  0%|          | 0/15 [00:00<?, ?it/s]

0
[5, 5, 17, 5, 5, 16, 3, 16, 0, 15, 12, 5, 13] QQWQQTDTASMQF
[16, 2, 18, 13, 3, 1, 13, 7, 3, 9, 2, 12, 18] TNYFDRFGDINMY
[16, 13, 2, 13, 3, 1, 13, 7, 3, 9, 2, 12, 18] TFNFDRFGDINMY
[16, 13, 18, 12, 3, 1, 13, 7, 3, 9, 2, 12, 18] TFYMDRFGDINMY
[13, 3, 12, 0, 15, 10, 13, 8, 17, 19, 19, 2, 17] FDMASLFHWVVNW
[6, 4, 5, 11, 7, 15, 12, 11, 13, 19, 11, 4, 17] ECQKGSMKFVKCW
[16, 13, 18, 13, 3, 1, 0, 7, 3, 9, 2, 12, 18] TFYFDRAGDINMY
[16, 13, 18, 13, 3, 1, 13, 10, 3, 9, 2, 12, 18] TFYFDRFLDINMY
[16, 13, 18, 13, 3, 1, 13, 7, 16, 9, 2, 12, 18] TFYFDRFGTINMY
[4, 10, 2, 14, 9, 14, 17, 9, 8, 16, 6, 19, 4] CLNPIPWIHTEVC
[16, 13, 18, 13, 3, 1, 13, 7, 3, 9, 4, 12, 18] TFYFDRFGDICMY
[16, 13, 18, 13, 3, 1, 13, 7, 3, 9, 2, 9, 18] TFYFDRFGDINIY
[12, 4, 8, 16, 13, 12, 0, 8, 9, 1, 11, 4, 18] MCHTFMAHIRKCY
predict models [0] recycles 6 hard 1 soft 0 temp 1 loss 3.95 i_con 3.89 plddt 0.38 ptm 0.91 i_ptm 0.39
core.import_pose.import_pose: File '././DOTS_peptide_4ib5_2024-04-27_08-06/QQWQQTDTASMQF.pdb' automatica



################################ End of Part III ################################

################################################################################

---------------------------------------------------------------------------- That's all folks ! ----------------------------------------------------------------------------


################################################################################